In [216]:
from collections import deque

In [479]:
bla = """8 3 11
1 2
1 3
2 3
3 4
4 5
5 6
5 7
5 8
4 6
3 7
7 8
12 5 1
11 10 6
1 6 7
3
3 4
4 4
7 5
"""

In [480]:
test = bla.split('\n')

info = test[0].split()

num_cities = int(info[0])
num_warehouses = int(info[1])
num_roads = int(info[2])

In [481]:
def path_exists(x: int, y:int):
    return (x,y) in connections or (y,x) in connections

def adj(x: int):
    return [b for a,b in connections if x == a] + [a for a,b in connections if x == b]

def dist_cities(start: int):
    dist = {} #distance from start to any node 
    visited = {}
    
    q = deque()
    q.append(start)
    dist[start] = 0
    
    while q:
        city = q.popleft() 
        if not city in visited or not visited[city]:
            visited[city] = True
            
            for z in adj(city): # follow all arcs
                q.append(z)
                if not z in dist:
                    dist[z] = dist[city] + 1
                    continue

                if z in dist:
                    curr_dist = dist[z]
                    if curr_dist > dist[city] + 1:
                        dist[z] = dist[city] + 1
            
    return dist

In [482]:
def lowest_cost_for_order(order):
    origin_city = order[1]
    num_orders = order[0]
    candidates = {k: v for k, v in dist_cities(origin_city).items() if k in warehouse_cities} 
    return map_costs(candidates)

def map_costs(candidates): # returns cost -> cost of transporting this one item
    map_candidates = {}
    for city, dist in candidates.items():
        warehouse_index = warehouse_dict[city]
        cost = warehouses[warehouse_index][1] * dist
        map_candidates[city] = cost
    
    return map_candidates

In [483]:
# tuple of (citya, cityb) connection
connections = []
for i in range(num_roads):
    connection = test[i+1].split()
    connections.append((int(connection[0]), int(connection[1])))
    
# list of (num_items, cost per km, city of warehouse)
warehouses_main = []
warehouse_cities = []
warehouse_dict = {}
for i in range(num_warehouses):
    warehouse = test[num_roads + 1 + i].split()
    warehouses_main.append([int(warehouse[0]), int(warehouse[1]), int(warehouse[2])])
    warehouse_dict[int(warehouse[2])] = i
    warehouse_cities.append(int(warehouse[2]))
    
total_orders = int(test[1 + num_roads + num_warehouses])

# tuple of (number of items, city of order)
orders_main = []
for i in range(total_orders):
    order = test[1 + 1 + num_roads + num_warehouses + i].split()
    orders_main.append([int(order[0]), int(order[1])])

In [484]:
def solve(orders, warehouses):
    total_cost = 0
    # get item from origin city first
    for idx, order in enumerate(orders):
        city_id = order[1]
        if city_id in warehouse_dict:
            idx = warehouse_dict[city_id]
            city = warehouses[idx]

            # city stock = order number just nice
            if city[0] == order[0]: 
                orders[idx][0] = 0
                warehouses[warehouse_dict[city_id]][0] = 0
                continue

            # if city has more stock than order
            if city[0] > order[0]:
                warehouses[warehouse_dict[city_id]][0] = city[0] - order[0]
                orders[idx][0] = 0
                continue

            # if city doesn't have enough stock for order
            if city[0] < order[0]:
                orders[idx][0] -= city[0]
                warehouses[warehouse_dict[city_id]][0] = 0
                continue
    
    # part 2
    
    for order in orders:
        while order[0] != 0:
            candidates = lowest_cost_for_order(order)

            city_id = -1
            city = [0,0]

            while (city[0] == 0):
                city_id = min(candidates, key=candidates.get)
                idx = warehouse_dict[city_id]
                city = warehouses[idx]
                if (city[0] == 0):
                    del candidates[city_id]

            # city stock = order number just nice
            if city[0] == order[0]: 
                total_cost += order[0] * candidates[city_id]
                order[0] = 0
                warehouses[warehouse_dict[city_id]][0] = 0

            # if city has more stock than order
            if city[0] > order[0]:
                total_cost += order[0] * candidates[city_id]
                warehouses[warehouse_dict[city_id]][0] = city[0] - order[0]
                order[0] = 0

            # if city doesn't have enough stock for order
            if city[0] < order[0]:
                total_cost += city[0] * candidates[city_id]
                order[0] -= city[0]
                warehouses[warehouse_dict[city_id]][0] = 0

    return total_cost

In [485]:
import copy, itertools

solns = []

for perm in list(itertools.permutations(orders_main)):
    ords = copy.deepcopy(perm)
    wars = copy.deepcopy(warehouses_main)
    solns.append(solve(ords, wars))

print(min(solns))

136
